In [1]:
import csv
import time
import os
import subprocess
import psutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


class CustomCallback(keras.callbacks.Callback):
    def __init__(self):
        super(CustomCallback, self).__init__()
        self.training_time = None
        self.average_utilization = None
        
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        self.utilizations = []
        
    def on_train_end(self, logs={}):
        self.training_time = time.time() - self.start_time
        self.average_utilization = np.mean(self.utilizations)
        print('\nTotal training time: {} seconds'.format(self.training_time))
        print('\nAverage GPU utilization during training so far: {}'.format(self.average_utilization))

    def on_epoch_end(self, epoch, logs={}):
        process = subprocess.Popen(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader'], stdout=subprocess.PIPE)
        utilization = int(process.stdout.readline().strip().decode('utf-8').split()[0])
        self.utilizations.append(utilization)

        
def Classify(batch_size, epochs):
    # Load the CIFAR10 dataset
    (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

    # Preprocess the data
    x_train = x_train / 255.0
    x_test = x_test / 255.0

    # Build the model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    print("\n◆Train:")
    callback = CustomCallback()
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), callbacks=[callback])
    print("Done")
    
    # Evaluate the model
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print("Test Accuracy:", test_acc)
    
    with open('data.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow([batch_size, epochs, callback.average_utilization, callback.training_time, test_loss, test_acc])

        
for batch in range(100, 801, 100):
    for epoch in range(10, 41, 10):
        Classify(batch, epoch)



2023-02-09 08:54:18.539253: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-09 08:54:18.560169: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-09 08:54:18.560372: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-09 08:54:18.561190: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags


◆Train:
Epoch 1/10


2023-02-09 08:54:20.709099: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8400
2023-02-09 08:54:21.147214: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-09 08:54:21.147564: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-09 08:54:21.147576: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-02-09 08:54:21.148020: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-09 08:54:21.148073: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


125/125 [==============================] - 3s 14ms/step - loss: 1.9523 - accuracy: 0.2724 - val_loss: 1.6032 - val_accuracy: 0.4260
Epoch 2/10
125/125 [==============================] - 1s 12ms/step - loss: 1.6331 - accuracy: 0.3962 - val_loss: 1.4873 - val_accuracy: 0.4619
Epoch 3/10
125/125 [==============================] - 1s 12ms/step - loss: 1.5222 - accuracy: 0.4483 - val_loss: 1.3378 - val_accuracy: 0.5178
Epoch 4/10
125/125 [==============================] - 2s 12ms/step - loss: 1.4414 - accuracy: 0.4830 - val_loss: 1.2791 - val_accuracy: 0.5470
Epoch 5/10
125/125 [==============================] - 1s 12ms/step - loss: 1.3597 - accuracy: 0.5179 - val_loss: 1.2079 - val_accuracy: 0.5729
Epoch 6/10
125/125 [==============================] - 2s 12ms/step - loss: 1.3088 - accuracy: 0.5391 - val_loss: 1.1895 - val_accuracy: 0.5750
Epoch 7/10
125/125 [==============================] - 1s 12ms/step - loss: 1.2558 - accuracy: 0.5557 - val_loss: 1.1160 - val_accuracy: 0.6053
Epoch 8/10